In [ ]:
import tensorflow as tf
from tensorflow import keras
import os

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

training_groundtruth = pd.read_csv('/kaggle/input/training-groundtruth/training-groundtruth-new.csv')
transcripted_text_path = '/kaggle/input/transcripted-text/'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
sentences = []
for filename in os.listdir(transcripted_text_path):
    if filename.endswith(".txt"):
        sentences.append(open(transcripted_text_path + filename, 'r').read())
vectorizer = CountVectorizer(min_df=0.05, max_df=0.9, lowercase=True)
vectorizer.fit(sentences)

In [ ]:
# vectorized_sentence = vectorizer.transform(sentences).toarray()

In [ ]:
data_train = []
data_test = []
x_train = []
x_test = []
y_train = []
y_test = []

for index, row in training_groundtruth.iterrows():
    if row['type'] == "train":
        data_train.append(row)
        y_train.append(row['dx'])
    else:
        data_test.append(row)
        y_test.append(row['dx'])
for rows_of_type in [data_train, data_test]:
    for row in rows_of_type:
        temp = []
        temp += [row['age']]
        temp += [1 if row['gender'] == 'female' else 0]
        temp += [row['educ'] if not np.isnan(row['educ']) else 0]
#         print(row['adressfname'])
#         print(open(transcripted_text_path + row['adressfname'] + '.txt', 'r').read())
        sentence = open(transcripted_text_path + row['adressfname'] + '.txt', 'r').read()
        sentence = vectorizer.transform([sentence]).toarray()
        temp = np.concatenate((temp, sentence[0]))

        if row['type'] == "train":
            x_train.append(temp)
        else:
            x_test.append(temp)
        break
    break

x_train = np.array(x_train)
x_test = np.array(x_test)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10)

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)